# SYMBA Model for QED:

We train this model to map the (amplitude) of particles interactions in quantum electrdynamics theory QED to the (squared amplitude), which is the key element in calculating the cross section.

# 

In [1]:
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization
import re

## Preparing the data:

To prepape the dataset for training, we use the symbolic computation program MARTY [7] to generate expressions for possible interactions in QED. We restrict the scope 2-to-2 and 2-to-3 particle tree-level processes. All interactions involving off-shell and on-shell particles, anti-particles and gauge bosons are included. Since it is possible for different amplitudes to yield the same squared expressions, we include such amplitudes in our dataset.  All output expressions (squared amplitudes) are simplified with the Python symbolic mathematics module SymPy [25].

You can find the data here:
https://drive.google.com/file/d/1SRsg0dPh0O2JLop2OsEfuml6qZVbaOFP/view?usp=share_link

In [2]:
with open('qed_data.txt', 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
    
text_pairs=[]
for line in lines[: min(len(lines), len(lines)-1)]:
    amplitude, squared_amplitude = line.split('>')
    text_pairs.append((amplitude, squared_amplitude ))


text_pairs = list(set(text_pairs))
print('data size: ', len(text_pairs))

data size:  134802


In [3]:
def sort(pairs):
    dic  = dict()
    for i,j in pairs:
        dic[(i,j)]=len(j)
    pairs_= list(sorted(dic.items(), key=lambda x: x[1]))
    
    pairs =[]
    for i,j in pairs_:
        pairs.append(i)
    return pairs

In [ ]:
text_pairs = sort(text_pairs)[:40000]

In [ ]:
len(text_pairs)

#### Examples:

In [4]:
print('Amplitude:  ' , '\n', text_pairs[1][0], '\n' )
print('Squared Amplitude:  ', '\n' ,text_pairs[1][1] )

Amplitude:   
 i* e^3* (m_t* gamma_{%\lambda,%eta,%eta} * gamma_{+%\lambda,%eps,%del} * gamma_{%\lambda,%eta,%del} * A_{l,+%\lambda} (p_1)* e_{j,%del} (p_3)* e_{k,%eps} (p_4)^(*)* t_{i,%del} (p_2)* t_{l,%eta} (p_5)^(*) + -p_1_%\sigma* gamma_{+%\sigma,%eta,%eta} * gamma_{%\lambda,%eta,%eta} * gamma_{+%\lambda,%eps,%del} * gamma_{%\lambda,%eta,%del} * A_{l,+%\lambda} (p_1)* e_{j,%del} (p_3)* e_{k,%eps} (p_4)^(*)* t_{i,%del} (p_2)* t_{l,%eta} (p_5)^(*) + p_5_%\sigma* gamma_{+%\sigma,%eta,%eta} * gamma_{%\lambda,%eta,%eta} * gamma_{+%\lambda,%eps,%del} * gamma_{%\lambda,%eta,%del} * A_{l,+%\lambda} (p_1)* e_{j,%del} (p_3)* e_{k,%eps} (p_4)^(*)* t_{i,%del} (p_2)* t_{l,%eta} (p_5)^(*)) /((s_33 + 2* s_34 + s_44)* (m_t^2 + -s_11 + 2* s_15 + -s_55)) 

Squared Amplitude:   
 [ m_t^4 , m_t^2 , 1] , [32 *( 2*m_e^2 + s_34) , 16 *( m_e^2 *( 4*s_11 + 4*s_12 - 4*s_15 - 3*s_25) + 2*s_11*s_34 + 4*s_13*s_24 + 4*s_14*s_23 - 2*s_15*s_34 - 3*s_23*s_45 - 3*s_24*s_35) , -16 *( m_e^2 *( s_11*s_25 - 2*s_12*s_15

### Data preprocessing:

In [5]:
#preprocessing for the amplitudes:

def prepro_ampl(data):

    for r in (('}', '} '),('{', '{'), (' + ',' + ' ), (' - ', ' - ') ,('*', '* '), ('(* )', '(*)'),('^', '^') , ('(', '('),(')', ')'),('/', ' /')  ,('  ', ' ') ) :  #,('{', ' {')
        data = data.replace(*r) 
        
    return data

#preprocessing for the squared amplitudes:

def prepro_squared_ampl(data):

    for r in (('*', '*'), (',', ' , '), ('*(', ' *(') , ('([', '[ '), ('])', ' ]'), ('[', '[ '), (']', ' ]'), ('[ start ]', '[start]'), ('[ end ]', '[end]') ,('/', ' / ') ,('  ', ' ')) :
        data = data.replace(*r) 
    data = re.sub(r"(\s\d+|\s-\d+|\s\+\d+)\*(s_\d+\*s_\d+)", r"\1* \2", data)
    data = re.sub(r"(\s\d+|\s-\d+|\s\+\d+)\*(m_\w+\^\d+)", r"\1* \2 ", data)
    data = re.sub(r"(m_\w+\^\d+)\(", r"\1 )", data)
    data = data.replace('  ', ' ')
    
    
    return data



text_pairs_prep = []
for i in range(len(text_pairs)):
    text_pairs_prep.append((prepro_ampl(text_pairs[i][0]), prepro_squared_ampl(text_pairs[i][1])))

text_pairs = text_pairs_prep

###  Maximum sequence length:

In [6]:
def max_len(sq_data):
    l = len(sq_data[sq_data.index(max(sq_data, key=len))].split())
    return l


ampl = [pair[0] for pair in text_pairs]
sq_ampl= [pair[1] for pair in text_pairs]

print( 'Maximum sequence length of amplitudes        :' ,max_len(ampl))
print( 'Maximum sequence length of squared amplitudes:' ,max_len(sq_ampl))

Maximum sequence length of amplitudes        : 180
Maximum sequence length of squared amplitudes: 152



# Tokenization:

### Split the data:

In [7]:
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs  = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

train_input_texts = [pair[0] for pair in train_pairs]
train_output_texts = [pair[1] for pair in train_pairs]

### Tokeniaztion:

In [8]:
sequence_length = max(max_len(ampl), max_len(sq_ampl))

input_vectorization = TextVectorization(
    max_tokens=None, output_mode="int", output_sequence_length=sequence_length, standardize=None, )

output_vectorization = TextVectorization(
    max_tokens=None,
    output_mode="int",
    output_sequence_length=sequence_length + 1, standardize=None)

input_vectorization.adapt(train_input_texts)
output_vectorization.adapt(train_output_texts)

target_tokens = output_vectorization.get_vocabulary()
input_tokens = input_vectorization.get_vocabulary()




print('number of input (amplitude) tokens:  ', len(input_tokens))
print('number of target (squared amplitude) tokens: ', len(target_tokens))

2022-11-23 12:43:15.237006: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


number of input (amplitude) tokens:   788
number of target (squared amplitude) tokens:  1424


In [9]:
batch_size = 64
vocab_size= max(len(input_tokens), len(target_tokens))

def format_dataset(input_exp, target_exp):
    input_exp = input_vectorization(input_exp)
    target_exp = output_vectorization(target_exp)
    return ({"encoder_inputs": input_exp, "decoder_inputs": target_exp[:, :-1],}, target_exp[:, 1:])


def make_dataset(pairs):
    ampl_texts, sqampl_texts = zip(*pairs)
    ampl_texts = list(ampl_texts)
    sqampl_texts = list(sqampl_texts)
    dataset = tf.data.Dataset.from_tensor_slices((ampl_texts, sqampl_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()



train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

## Transformer Model:

In [10]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int64")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int64")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int64")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)

In [ ]:

    
embed_dim = 256
latent_dim = 512
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)



## Training:

In [ ]:
epochs = 5  
learning_rate=0.001
opt = keras.optimizers.Adam(learning_rate=learning_rate)

transformer.summary()
transformer.compile(loss="sparse_categorical_crossentropy", metrics=["accuracy"], optimizer=opt)
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)

## Inference:

In [ ]:
target_tokens = output_vectorization.get_vocabulary()
target_index_lookup = dict(zip(range(len(target_tokens)), target_tokens))
max_decoded_sentence_length = max_sequence_length


def decode_sequence(input_sentence):
    tokenized_input_sentence = input_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = output_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = target_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence

test_input_texts = [pair[0] for pair in test_pairs]
test_output_texts = [pair[1] for pair in test_pairs]



In [ ]:
for i in random.sample(range(0,len(test_input_texts)), 5):
    input_sentence = test_input_texts[i]
    translated = decode_sequence(input_sentence)
    print('Actual:    ', test_output_texts[i], '\n')
    print('Predicted: ', translated, '\n')
    print(test_output_texts[i]==translated, '\n')